<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap
import os

In [ ]:
geemap.show_youtube("_6JOA-iiEGU")

In [ ]:
Map = geemap.Map()
Map

## Download an ee.Image

In [ ]:
image = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 1.4}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 0.7)

In [ ]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                [-115.413031, 35.889467],
                [-115.413031, 36.543157],
                [-114.034328, 36.543157],
                [-114.034328, 35.889467],
                [-115.413031, 35.889467],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
filename = os.path.join(out_dir, "landsat.tif")

### Exporting all bands as one single image

In [ ]:
image = image.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=False
)

### Exporting each band as one image

In [ ]:
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=True
)

### Export an image to Google Drive

In [ ]:
geemap.ee_export_image_to_drive(
    image, description="landsat", folder="export", region=roi, scale=30
)

## Download an ee.ImageCollection

In [ ]:
import ee
import geemap
import os

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(loc)
    .filterDate("2008-01-01", "2020-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")

In [ ]:
print(collection.aggregate_array("system:index").getInfo())

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder="export", scale=10)

## Extract pixels as a Numpy array

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810").select(["B4", "B5", "B6"])

aoi = ee.Geometry.Polygon(
    [[[-110.8, 44.7], [-110.8, 44.6], [-110.6, 44.6], [-110.6, 44.7]]], None, False
)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255 * ((rgb_img[:, :, 0:3] - 100) / 3500)).astype("uint8")
plt.imshow(rgb_img_test)
plt.show()